In [1]:
import pandas as pd
import numpy as np

import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from time import sleep
from random import uniform

import concurrent.futures
from functools import partial, reduce    

from yahoofinancials import YahooFinancials
from warnings import warn
from datetime import date, timedelta

import scipy.optimize as sco

# Descarga de datos

In [2]:
def __cryptos__(top: int) -> list:
    '''
    Esta función extrae por grupos de 100, los tickers de cryptomonedas en yahoo finance organizadas por marketcap
    '''
    #Url a consultar
    search_url='https://finance.yahoo.com/cryptocurrencies?count=100&offset={}'.format(top)

    #Se abre la página web y esperamos un rato para no ser sospechosos
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(search_url)
    sleep(uniform(0, 1))

    #Busca la tabla de la página
    table_path = './/table'
    tabla = driver.find_element_by_xpath(table_path)

    #Se queda con los elementos únicos de la tabla
    arr = [text.split(' ') for text in tabla.text.split('\n')]
    cryptos = np.squeeze([element for element in arr if len(element) is 1][1:])

    #Cierra la página
    driver.quit()
    
    return cryptos

def top_cryptos(top:int) -> list:
    '''
    Esta función extrae los tickers del top N de cryptomonedas con mayor marketcap en yahoo finance.
    '''
    #Mapea la función que trae cada 100 cryptos en un rango de 0 a top
    map_cryptos = map(__cryptos__, range(0, top, 100))
    #Desempaqueta el mapa
    arr = [*map_cryptos]
    #Aplana la lista
    cryptos = [item for sublist in arr for item in sublist]
    
    return cryptos

def extraction(ticker: object, 
               sdate: object,
               edate: object,
               frequency: object,
               volatility: bool = False) -> pd.DataFrame:
    '''
    Esta función extrae el ticker con las fechas y la frecuencia especificadas. Devuelve un dataframe con
    tres columnas: la fecha, los precios de cierre (precio ajustado), y una medida de volatilidad del periodo.
    '''
    #Consulta el ticker
    stock = YahooFinancials(ticker)
    #Se trae las fechas históricas con la frecuencia requerida
    price = stock.get_historical_price_data(start_date = sdate, 
                                end_date = edate, 
                                time_interval = frequency)

    #Toma los precios
    df_price = pd.DataFrame(price[ticker]['prices'])
    #Intenta calcular precio y volatilidad
    try:
        #Toma el precio de cierre ajustado
        df_price['{}_price'.format(ticker)] = df_price['adjclose']
        if volatility is True:
            #Con precios máximos, mínimos y cierre calcula volatilidad
            df_price['{}_volatility'.format(ticker)] = (df_price['high'] - df_price['low'])/df_price['close']
    #Si no lo logra devuelve Null
    except:
        warn('Error en la moneda {}'.format(ticker))
        df_price['{}_price'.format(ticker)] = None
        if volatility is True:
            #Con precios máximos, mínimos y cierre calcula volatilidad
            df_price['{}_volatility'.format(ticker)] = None
        
        df_price['formatted_date'.format(ticker)] = None
    #Al final selecciona las columnas finales
    finally:
        #Formatea fecha
        df_price.drop(df_price.columns.difference(['formatted_date',
                                               '{}_price'.format(ticker),
                                               '{}_volatility'.format(ticker)]), 1, inplace=True)
    
    return df_price

In [3]:
fecha_inicial = '2015-01-01'
fecha_final = (date.today() - timedelta(days=1)).strftime("%Y-%m-%d")
top = 300
freq = 'daily' 

#Extrae los primeros 500 tickers con mayor capitalización en yahoo
tickers = top_cryptos(top)

#Fija los argumentos extra de la función
extract_ticker = partial(extraction,
                         sdate = fecha_inicial,
                         edate = fecha_final,
                         frequency = freq,
                         volatility = False)

#Lo mapea en un iterador para no gastar tanta memoria y trae históricos de 
#los 500 tickers
with concurrent.futures.ThreadPoolExecutor() as executor:
    all_cryptos = executor.map(extract_ticker, tickers)

#Reduciremos el mapa de data frames en un data 
cryptos_reduce = partial(pd.merge, on='formatted_date', how='outer')                                                              
#Reduce el mapa de dataframes en un solo dataframe
df = reduce(cryptos_reduce, all_cryptos) 

#Se trae la tasa "libre de riesgo"
df_rf = extraction('^TNX', fecha_inicial, fecha_final, freq).dropna()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 99.0.4844
[WDM] - Get LATEST driver version for 99.0.4844


[WDM] - Driver [C:\Users\david\.wdm\drivers\chromedriver\win32\99.0.4844.51\chromedriver.exe] found in cache
[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 99.0.4844
[WDM] - Get LATEST driver version for 99.0.4844
[WDM] - Driver [C:\Users\david\.wdm\drivers\chromedriver\win32\99.0.4844.51\chromedriver.exe] found in cache


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 99.0.4844
[WDM] - Get LATEST driver version for 99.0.4844


[WDM] - Driver [C:\Users\david\.wdm\drivers\chromedriver\win32\99.0.4844.51\chromedriver.exe] found in cache
C:\Users\david\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:66: UserWarning: Error en la moneda WTRX-USD
C:\Users\david\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:66: UserWarning: Error en la moneda ICP-USD
C:\Users\david\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:66: UserWarning: Error en la moneda BTT2-USD
C:\Users\david\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:66: UserWarning: Error en la moneda GLMR-USD
C:\Users\david\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:66: UserWarning: Error en la moneda DESO-USD
C:\Users\david\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:66: UserWarning: Error en la moneda GMT3-USD
C:\Users\david\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py

# Preprocesamiento

In [4]:
thr_nulls = 0.5

#Depuramos los datos
df_filter = df.loc[:, df.isnull().sum(axis = 0)/df.shape[0] < thr_nulls].copy()
df_filter.dropna(axis = 0, how = 'any', inplace = True)

#Fijamos índices
df_filter = df_filter.set_index(pd.to_datetime(df_filter['formatted_date']))
df_filter.drop(columns = ['formatted_date'], inplace = True)

df_rf = df_rf.set_index(pd.to_datetime(df_rf['formatted_date']))
df_rf.drop(columns = ['formatted_date'], inplace = True)

#Se calculan cambios porcentuales
df_prices = df_filter.copy()
df_filter = df_filter.apply(lambda x: x.pct_change(), axis = 0)
df_filter.dropna(inplace = True)

# Media varianza

In [5]:
def portfolio_std_mean(rt: np.array, cov: np.array, w: np.array, rf: float, df: pd.DataFrame) -> tuple:
    '''
    Calcula media, covarianza y sharpe ratio del portafolio
    '''
    #Rentabilidad del portafolio anualizada
    p_rt = np.dot(w.T, rt)*252
    #Matriz de varianza y covarianza
    p_cov = np.sqrt(np.dot(w.T, np.dot(cov, w)))*np.sqrt(252)
    #Sharpe ratio
    p_sharpe = (p_rt-rf)/p_cov
    
    zip_w = zip(df.columns, [w_i.item() for w_i in w])
    weights = dict(zip_w)

    return p_rt.item(), p_cov.item(), p_sharpe.item(), weights


def __variance__(w: np.array, rt: np.array, cov: np.array, rf: float, df: pd.DataFrame) -> float:
    '''
    Retorna la varianza del portafolio (función objetivo)
    '''
    #Varianza del portafolio (función a minimizar)
    _, var, _, _ = portfolio_std_mean(rt, cov, w, rf, df)
    return var



def __neg_sharpe_ratio__(w: np.array, rt: np.array, cov: np.array, rf: float, df: pd.DataFrame) -> float:
    '''
    Retorna el sharp ratio negativo (función objetivo)
    '''
    #Sharpe ratio del portafolio (función a minimizar)
    _, _, sharpe, _ = portfolio_std_mean(rt, cov, w, rf, df)
    return -sharpe



def optimal_portfolio(df: pd.DataFrame, rf: float,
                      typ: str = 'sharpe') -> tuple:
    '''
    Calcula el portafolio de Sharpe ratio máximo o de varianza mínima
    '''
    #Rentabilidad del portafolio
    rt = df.mean(axis = 0).values.reshape(df.shape[1], 1)
    #Covarianza del portafolio
    cov = df.cov().values
    
    #Cantidad de activos
    num_assets = rt.shape[0]
    #Argumentos secundarios de la función a minimizar
    args = (rt, cov, rf, df)
    #Restricción: en total debe sumar el 100%
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
        
    #Cada ponderador debe estar en 0 y 100
    bound = (0.0,1.0)
    #y debe estar para cada activo
    bounds = tuple(bound for asset in range(num_assets))
    
    #Se inicializa de forma aleatoria cada ponderador, todos igual
    initial_w = num_assets*[1/num_assets]
    
    #Se elige si se maximiza el sharpe ratio o si se minimiza la varianza
    if typ == 'sharpe':
        obj_fun = __neg_sharpe_ratio__
    elif typ == 'var':
        obj_fun = __variance__
    
    #Se aplica la minimización según las restricciones
    result = sco.minimize(obj_fun, initial_w, args=args,
                          method='SLSQP', bounds=bounds, constraints=constraints)
    
    #Se calcula la rentabilidad del portafolio, junto a su volatilidad y los ponderadores óptimos
    optimal_result = portfolio_std_mean(rt, cov, result['x'].reshape(num_assets, 1), rf, df)
    
    return optimal_result


def opt_portfolio_date(day: int, 
                       df: pd.DataFrame,
                       df_rf: pd.DataFrame,
                       window:int = 90,
                       min_inv: float = 0.01,
                       typ: str = 'sharpe') -> dict:
    '''
    Se calcula el portafolio óptimo (sharpe o varianza mínima) hasta el día day con una ventana de tiempo window,
    dejando los activos que tienen una participación superior a min_inv
    '''
    #Si la ventana fuera mayor al día se desorganizaría el datafrma
    if window > day:
        raise ValueError('Ventana de tiempo superior al periodo desde el que empezaría')
        
    #Fecha desde la que se tendrá en cuenta el yield del bono a 10 años
    date_rf_since = df_rf.index >= df.index[(day-window):day][1]
    #Fecha hasta la que se tendrá en cuenta el yield del bono a 10 años
    date_rf_until = df_rf.index <= df.index[(day-window):day][-1]
    #Yield promedio para este periodo
    rf = df_rf[date_rf_since & date_rf_until].mean()[0]/100
    
    #Aquí se define si se calculara portafolio de varianza mínima o varianza con sharpe ratio máximo
    if typ == 'sharpe':
        result = optimal_portfolio(df.iloc[(day-window):day, :], rf, 'sharpe')
    elif typ == 'var':
        result = optimal_portfolio(df.iloc[(day-window):day, :], rf, 'var')
    
    #Aquí se filtran solo los activos cuya participación indica superior al min_inv%
    opt_port = {ticker:value for ticker, value in result[3].items() if value > min_inv}
    
    return {'date': df.index[day], 'portfolio': opt_port}

In [6]:
fecha_desde = 120
fecha_hasta = df_filter.shape[0]

#Maximiza sharpe ratio con inversión mínima de 1% y una ventana de 90 días
partial_opt_portfolio = partial(opt_portfolio_date, df=df_filter, df_rf=df_rf,
                                window = 120, 
                                min_inv = 0.01,
                                typ = 'sharpe')   

#Se hace concurrencia para agilizar
with concurrent.futures.ThreadPoolExecutor() as executor:
    all_dicts = executor.map(partial_opt_portfolio, range(fecha_desde, fecha_hasta))

#Se preprocesa para dejar las participaciones como dataframe
arr_alloc =[*all_dicts]
df_alloc = pd.DataFrame(arr_alloc)
df_alloc = pd.concat([df_alloc['date'], pd.json_normalize(df_alloc['portfolio'])], axis = 1).fillna(0).copy()

#Fijamos índices
df_alloc = df_alloc.set_index(pd.to_datetime(df_alloc['date']))
df_alloc.drop(columns = ['date'], inplace = True)

#Ajustamos para asignar el 100% de la plata
df_alloc[df_alloc > 0] = df_alloc + np.array((1 - df_alloc.sum(axis = 1))/np.sum(df_alloc > 0, axis = 1)).reshape(-1, 1)

In [7]:
arr_alloc[-1]

{'date': Timestamp('2022-03-14 00:00:00'),
 'portfolio': {'WAVES-USD_price': 0.052012197823109646,
  'SYS-USD_price': 0.2551558598869126,
  'POWR-USD_price': 0.23365465190418352,
  'CEEK-USD_price': 0.06020316579749514,
  'REQ-USD_price': 0.12610588999964484,
  'VERI-USD_price': 0.20052014433210605,
  'CENNZ-USD_price': 0.07200305311078642}}

# Análisis del portafolio

In [ ]:
env = flappy_bird_gym.make("FlappyBird-v0")

In [9]:
import random
import numpy as np
from collections import deque

from keras.layers import Input, Dense
from keras.models import load_model, save_model, Sequential
from keras.optimizers import RMSprop

#Neural Network for Agent
def NeuralNetwork(input_shape, output_shape):
    model = Sequential()
    model.add(Dense(512, input_shape=input_shape, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(output_shape, activation='linear', kernel_initializer='he_uniform'))
    model.compile(loss='mse', optimizer=RMSprop(lr=0.0001, rho=0.95, epsilon=0.01), metrics=['accuracy'])
    model.summary()
    return model


ModuleNotFoundError: No module named 'tensorflow'

In [32]:
df_alloc.head()

,TUSD-USD_price,NEXO-USD_price,FLUX-USD_price,XYO-USD_price,REV1-USD_price,DOGE-USD_price,WAVES-USD_price,REN-USD_price,XRP-USD_price,CEEK-USD_price,...,XEM-USD_price,LRC-USD_price,XMR-USD_price,CVC-USD_price,DCR-USD_price,POLY-USD_price,ADA-USD_price,DENT-USD_price,BTG-USD_price,ELF-USD_price
date,,,,,,,,,,,,,,,,,,,,,
2018-12-02,0.714040,0.104982,0.093135,0.087842,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-12-03,0.732209,0.094313,0.089358,0.084121,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-12-04,0.699698,0.114396,0.080792,0.105115,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-12-05,0.725347,0.111640,0.054575,0.108438,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-12-06,0.657411,0.099748,0.074771,0.168070,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
prueba = df_alloc.copy()
#prueba.columns
indices = prueba > 0
for i in range((prueba.shape[0]-200), prueba.shape[0]):
    print(prueba.columns[indices.iloc[i]])

Index(['XYO-USD_price', 'DERO-USD_price', 'DOGE-USD_price', 'DAG-USD_price',
       'MVL-USD_price', 'RLC-USD_price', 'TEL-USD_price'],
      dtype='object')
Index(['XYO-USD_price', 'DERO-USD_price', 'DOGE-USD_price', 'DAG-USD_price',
       'RLC-USD_price', 'TEL-USD_price'],
      dtype='object')
Index(['XYO-USD_price', 'DERO-USD_price', 'DOGE-USD_price', 'DAG-USD_price',
       'RLC-USD_price', 'IOTX-USD_price', 'TEL-USD_price'],
      dtype='object')
Index(['XYO-USD_price', 'DERO-USD_price', 'DOGE-USD_price', 'DAG-USD_price',
       'MVL-USD_price', 'RLC-USD_price', 'REQ-USD_price', 'IOTX-USD_price',
       'TEL-USD_price'],
      dtype='object')
Index(['XYO-USD_price', 'DERO-USD_price', 'DOGE-USD_price', 'DAG-USD_price',
       'RLC-USD_price', 'REQ-USD_price', 'IOTX-USD_price', 'TEL-USD_price'],
      dtype='object')
Index(['XYO-USD_price', 'DERO-USD_price', 'DAG-USD_price', 'RLC-USD_price',
       'REQ-USD_price', 'IOTX-USD_price', 'TEL-USD_price'],
      dtype='object')
Index(['

# Simulaciones

In [42]:
def sim_investing(entry_pos, entry_date, exit_date,
                 df_prices, df_alloc):
    '''
    Calcula los beneficios y la posición final luego da haberse quedado durante un tiempo
    '''
    #Nos interesan los precios que podemos tomar
    df_prices = df_prices.loc[df_prices.index.isin(df_alloc.index), df_alloc.columns]
    
    #Se le fecha de entrada es luego de la de salida se igualan ambas fechas
    if entry_date > exit_date:
        entry_date = exit_date
    
    #ingresos
    earnings = np.sum(entry_pos*df_alloc.iloc[entry_date, :]*(df_prices.iloc[exit_date, :]/df_prices.iloc[entry_date, :]))
    #costos
    costs = 0.001*entry_pos + np.max([0.035*entry_pos, 10])
    #beneficios
    profits = earnings - costs
    
    return profits, exit_date

In [ ]:
init_entry_pos = 1000

step = model.train(x[0,:])
entry_pos, entry_date = sim_investing(init_entry_pos, 0, 0 + step, df_prices, df_alloc)

while entry_date < df_alloc.shape[0]
    step = model.train(x[step,:])
    entry_pos, entry_date = sim_investing(entry_pos, entry_date, entry_date + step, df_prices, df_alloc)
    
final_profit = entry_pos

In [ ]:
step = model.predict(x)
entry_pos, entry_date = sim_investing(entry_pos, entry_pos, entry_date + step, df_prices, df_alloc)